## Dependencies

In [ ]:
!pip install --pre -I "weaviate-client==4.*"

In [ ]:
!pip install weaviate-demo-datasets

## Connect to a local instance

In [ ]:
import weaviate, os, json
import weaviate.classes as wvc

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": os.environ['OPENAI_API_KEY']  # Replace with your inference API key
    }
)

print(client.is_ready())

## Create the base collections

In [ ]:
if (client.collections.exists("JeopardyCategory")):
    client.collections.delete("JeopardyCategory")

if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyCategory",
    description="A Jeopardy! category",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.Configure.Generative.openai(),
    properties=[
        wvc.Property(name="title", data_type=wvc.DataType.TEXT),
    ]
)

# CrossRefDefinition
client.collections.create(
    name="JeopardyQuestion",
    description="A Jeopardy! question",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.Configure.Generative.openai(),
    properties=[
        wvc.Property(name="question", data_type=wvc.DataType.TEXT),
        wvc.Property(name="answer", data_type=wvc.DataType.TEXT),
        # highlight-start
        wvc.ReferenceProperty(
            name="hasCategory",
            target_collection="JeopardyCategory"
        )
        # highlight-end
    ]
)

client.collections.list_all()

## Import data

In [ ]:
import weaviate_datasets
dataset = weaviate_datasets.JeopardyQuestions1k()  # instantiate dataset

old_client = weaviate.Client("http://localhost:8080")
dataset = weaviate_datasets.JeopardyQuestions10k()  # instantiate dataset
dataset.upload_objects(old_client, 100)

## Test example

In [ ]:
jeopardy = client.collections.get("JeopardyQuestion")
result = jeopardy.query.fetch_objects(limit=10)

# print result properties 
for o in result.objects:
    print(o.properties)